In [17]:
# import library
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_upstage import ChatUpstage, UpstageEmbeddings, UpstageGroundednessCheck
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.documents import Document
from dotenv import load_dotenv
import glob
import os

### Pinecone Index 생성

In [18]:
load_dotenv()
api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=api_key)

In [19]:
index_name = "quickstart"
if index_name not in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=4096, 
        metric="dotproduct",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ) 
    )
    print(f"{index_name} has been successfully created")
else:
    print(f"{index_name} is already exists.")

quickstart is already exists.


In [20]:
print(pc.list_indexes())

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 4096,
              'host': 'quickstart-e04boic.svc.aped-4627-b74a.pinecone.io',
              'metric': 'dotproduct',
              'name': 'quickstart',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


### Pinecone DB에 데이터 넣기

In [7]:
llm = ChatUpstage(api_key=os.environ.get("UPSTAGE_API_KEY"), temperature=0)
embeddings_passage = UpstageEmbeddings(model="embedding-passage") #4096
embeddings_query = UpstageEmbeddings(model="embedding-query") #4096
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)

split_docs = []
files = sorted(glob.glob("data/*.pdf"))

for file in files:
    loader = PyMuPDFLoader(file)
    split_docs.extend(loader.load_and_split(text_splitter))

# 문서 개수 확인
len(split_docs)

84

In [8]:
def preprocess_docs(docs, keys = ['source', 'page'], min_len=10):
    result_docs = []
    for doc in docs:
        doc.metadata = {key: doc.metadata[key] for key in keys}
        if len(doc.page_content) > min_len:
            result_docs.append(doc)
    return result_docs

split_docs = preprocess_docs(split_docs)

In [9]:
# 불용어
f = open("불용어.txt", 'r') # https://www.ranks.nl/stopwords/korean
stopwords = [word.rstrip() for word in list(f)]

# 불용어 처리하는 로직 필요.
def remove_stopwords(docs):
    return docs

In [10]:
docsearch = PineconeVectorStore.from_documents(split_docs, embeddings_passage, index_name=index_name)

### Search query 보내기

In [21]:
query = "소수림왕이 한 일은?"

query_vector = embeddings_query.embed_query(query)

client = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"), source_tag="langchain")
index = client.Index(index_name)

results = index.query(vector = query_vector, top_k = 10, include_metadata=True)
docs = []
for res in results["matches"]:
    metadata = res["metadata"]
    id = res.get("id")
    if "text" in metadata:
        text = metadata.pop("text")
        score = res["score"]
        docs.append(
            (Document(id=id, page_content=text, metadata=metadata), score)
        )
    else:
        print(f"Found document with no text key. Skipping.")

In [22]:
docs

[(Document(id='8dbfa848-8570-485a-8741-3116e671f25a', metadata={'page': 16.0, 'source': 'data/한국사미리보기.pdf'}, page_content='하였고, 2세기 후반 고국천왕 사후 왕위 계승 분쟁을 고비로 왕위 부자 \n계승 원칙이 확고해졌다. 4세기 후반 소수림왕은 태학을 설립하여 유학\n을 장려하였으며, 율령\n●을 반포하여 통치 조직을 새롭게 정비하였다.\n4세기 말 광개토 대왕은 남쪽으로 백제를 압박한 후 신라에 침입한 \n왜를 격퇴하였고, 북쪽으로는 후연과 거란을 물리쳐 영토를 넓혔다.  \n5세기 초 장수왕은 도읍을 평양으로 옮기고, 백제의 수도 한성을 점령\n한 후 한강 유역을 차지하였다(475). 고구려의 영토 확장 과정은 광개\n토 대왕릉비와 충주 고구려비에 잘 나타나 있다.\n백제가 한강 유역을 중심으로 주도권을 잡다\n백제는 한강 유역의 지리적 이점과 철기 문화를 기반으로 성장하였\n다. 3세기 중반 고이왕 때 좌평 관제를 마련하고 관복을 제정하는 등 \n국가 조직을 정비하였다. 4세기 중반 근초고왕은 남으로 세력을 넓혀 \n마한의 여러 소국을 복속해 나갔고, 고구려의 평양성을 공격하였으며'),
  0.456811666),
 (Document(id='9ee2eeba-e1a3-44f5-a7a3-ce7b537e938e', metadata={'page': 16.0, 'source': 'data/한국사미리보기.pdf'}, page_content='하였고, 2세기 후반 고국천왕 사후 왕위 계승 분쟁을 고비로 왕위 부자 \n계승 원칙이 확고해졌다. 4세기 후반 소수림왕은 태학을 설립하여 유학\n을 장려하였으며, 율령\n●을 반포하여 통치 조직을 새롭게 정비하였다.\n4세기 말 광개토 대왕은 남쪽으로 백제를 압박한 후 신라에 침입한 \n왜를 격퇴하였고, 북쪽으로는 후연과 거란을 물리쳐 영토를 넓혔다.  \n5세기 초 장수왕은 도읍을 평양으로 옮기고, 백제의 수도 한성을 점령\n한

In [25]:
response_llm = llm.invoke(query)
response_llm.content

'소수림왕은 고구려의 제15대 왕으로, 371년부터 384년까지 재위했습니다. 그의 주요 업적 중 하나는 불교를 고구려에 전파한 것입니다. 소수림왕은 불교를 받아들여 국가의 안정과 번영을 도모하고자 했습니다. 이를 위해 그는 불교 사찰인 초문사(初門寺)를 건립하고, 불교 경전을 번역하여 보급하는 등의 노력을 기울였습니다.\n\n또한 소수림왕은 군사적인 정복 활동을 통해 고구려의 영토를 확장했습니다. 그는 백제와 신라와의 전투에서 승리를 거두며 고구려의 국력을 강화했습니다. 이를 통해 고구려는 동북아시아의 강대국으로 성장하게 되었습니다.\n\n소수림왕은 고구려의 문화와 예술에도 큰 관심을 가졌습니다. 그는 예술과 학문을 장려하여 고구려의 문화적인 발전을 촉진했습니다. 그의 통치는 고구려의 역사에서 중요한 시기로 평가되고 있습니다.'

In [26]:
prompt = f"Question: {query}\n 사전지식: {docs[0][0].page_content}\n 사전 지식을 활용해서 Question에 대한 답변을 해줘"
response_llm_with_RAG = llm.invoke(prompt)
response_llm_with_RAG.content

'소수림왕은 4세기 후반에 고구려의 왕으로, 태학을 설립하여 유학을 장려하고, 율령을 반포하여 통치 조직을 새롭게 정비하였습니다.'

In [29]:
groundedness_check = UpstageGroundednessCheck()
request_input = {
    "context": response_llm_with_RAG.content,
    "answer": response_llm.content,
}
response = groundedness_check.invoke(request_input)
print(response)

grounded


### hybrid retrieval
https://arxiv.org/abs/2210.11934

https://docs.pinecone.io/guides/data/understanding-hybrid-search

serverless 환경과 pod-based 환경 사이에 정확도 차이가 있을 수 있음. 
 - serverless: query planner가 dense vecotr의 유사성을 기준으로 cluster를 선택 후, dense, sparse 고려해서 다시 선택.
 - pod-based는 처음부터 둘다 고려하나 봄..